### Imports, defining the decorator, ...

In [5]:
import pandas as pd
from marquez_client import MarquezClient
from marquez_client.models import JobType
import os
from openlineage.client import OpenLineageClient
from openlineage.client.transport.file import FileConfig, FileTransport
from openlineage.client.event_v2 import RunEvent, RunState, Run, Job, Dataset, DatasetEvent, JobEvent
from openlineage.client.uuid import generate_new_uuid
from datetime import datetime
from openlineage.client.facet import NominalTimeRunFacet
import logging
from functools import wraps
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import pickle
import re
# import brightway2 as bw
import bw2data as bd
import bw2io as bi
# import bw2calc as bc
# import bw2analyzer as bwa
import inspect
import getpass
from openlineage.client.uuid import generate_new_uuid

In [6]:
bi.__version__

'0.9.DEV38'

In [3]:
# bi.ecoinvent.import_ecoinvent_release('3.10', 'cut-off')

In [7]:
def log_job_2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):

        if not args and not kwargs:
            return func

        transport_type = "http"

        if transport_type == "file":
            file_config = FileConfig(
                log_file_path="test-lineage-file",
                append=True,
            )
            client = OpenLineageClient(transport=FileTransport(file_config))
        elif transport_type == "http":
            http_config = HttpConfig(
                url="http://localhost:5000",
                endpoint="api/v1/lineage",
                timeout=5,
                verify=False,
                # auth=ApiKeyTokenProvider({"apiKey": "f048521b-dfe8-47cd-9c65-0cb07d57591e"}),
                compression=HttpCompression.GZIP,
            )
            client = OpenLineageClient(transport=HttpTransport(http_config))
        elif transport_type == "console":
            # See https://openlineage.io/docs/client/python#console
            console_config = ConsoleConfig()
            client = OpenLineageClient(transport=ConsoleTransport(console_config))


        description = func.__doc__
        func_name = func.__name__
        executor = getpass.getuser() ### will this surely run on other systems?

        global last_params
        sig = inspect.signature(func)
        bound_args = sig.bind(*args, **kwargs)
        bound_args.apply_defaults()

        last_params = bound_args.arguments  # Store the last parameters
        # Define the job (same for all function executions)
        job = Job(namespace="my_namespace_3", name="data_processing_job_3")

        run = Run(
            runId=str(generate_new_uuid()),
            facets={
                "transformation_description": description, #BaseFacet(description=description)
                'name' : func_name,
                'executed by': executor,
                'arguments': last_params
            }  )


        if args:
            input_data = args[0] 
            output_data = func(*args, **kwargs)

        else:
            input_data = None 
            output_data = func(**kwargs)

            

        # input_dataset = Dataset(
        #     namespace="my_namespace_3",
        #     name=input_name,
        #     facets={} )

        # output_dataset = Dataset(
        #     namespace="my_namespace_3",
        #     name=output_name,
        #     facets={} )

        client.emit(
            RunEvent(
                eventType=RunState.COMPLETE,
                eventTime=datetime.now().isoformat(),
                run=run,
                job=job,
                inputs=[input_data.to_dict() if ((type(input_data)) == pd.core.frame.DataFrame) | (type(input_data)== pd.core.frame.Series) else input_data],
                outputs=[output_data.to_dict() if ((type(output_data)) == pd.core.frame.DataFrame) | (type(output_data)== pd.core.frame.Series) else output_data],
            )    )

        return output_data
        wrapper.__name__ = func.__name__ + "_decorated" ### to keep the name of the function as is

    return wrapper

In [20]:
df_1 = pd.read_csv('technosphere exchanges description.csv')

In [21]:
@log_job_2
def times_two(input = 154):
    '''multiplies the input by 2'''
    output = input * 2
    return output

In [22]:
df_2 = times_two(df_1)

In [19]:
# times_two().__name__ , times_two.__name__ ### WHY does this happen?--> fixed for the moment

('times_two', 'times_two')

In [20]:
@log_job_2
def first_forward(input):
    '''removes the first column'''
    if type(input) == pd.core.frame.DataFrame:
        output = input.iloc[:, 1:]
    return output

In [22]:
df_3 = first_forward(df_2)

In [23]:
os.environ['OPENLINEAGE_CONFIG'] = 'openlineage.yml'  

### Lets apply it to ecoinvent

In [15]:
from inspect import getmembers, isfunction

# print(getmembers(bi, isfunction))

[('add_example_database', <function add_example_database at 0x0000022BBCA5F7E0>), ('backup_data_directory', <function backup_data_directory at 0x0000022B9F39A7A0>), ('backup_project_directory', <function backup_project_directory at 0x0000022B9F39A840>), ('bw2setup', <function bw2setup at 0x0000022BBF2EC720>), ('create_core_migrations', <function create_core_migrations at 0x0000022BBCA5F880>), ('create_default_biosphere3', <function create_default_biosphere3 at 0x0000022BBD55EB60>), ('create_default_lcia_methods', <function create_default_lcia_methods at 0x0000022BBCA5E200>), ('es2_activity_hash', <function es2_activity_hash at 0x0000022BBCE831A0>), ('get_csv_example_filepath', <function get_csv_example_filepath at 0x0000022B9F2D65C0>), ('get_xlsx_example_filepath', <function get_xlsx_example_filepath at 0x0000022B9F2D6700>), ('import_ecoinvent_release', <function import_ecoinvent_release at 0x0000022BBF279DA0>), ('install_project', <function install_project at 0x0000022BBCB46480>), ('k

In [24]:
import importlib

importlib.reload(bi) ### in case you want to have a fresh start for bw2io

# bi.SimaProLCIACSVImporter().__doc__

<module 'bw2io' from 'c:\\miniconda\\envs\\brightcon\\Lib\\site-packages\\bw2io\\__init__.py'>

In [17]:
# for name, obj in inspect.getmembers(bi):
#     print(name), print(inspect.isfunction(obj))

In [18]:
# for method_name, method in inspect.getmembers(bi.SimaProCSVImporter):
#     print(method)

<class 'type'>
<slot wrapper '__delattr__' of 'object' objects>
{'__module__': 'bw2io.importers.simapro_csv', 'format': 'SimaPro CSV', '__init__': <function SimaProCSVImporter.__init__ at 0x0000022BBCB46020>, 'get_db_name': <function SimaProCSVImporter.get_db_name at 0x0000022BBCB460C0>, 'write_database': <function SimaProCSVImporter.write_database at 0x0000022BBCB46160>, 'match_ecoinvent2': <function SimaProCSVImporter.match_ecoinvent2 at 0x0000022BBCB46200>, '__doc__': None}
<method '__dir__' of 'object' objects>
None
<slot wrapper '__eq__' of 'object' objects>
<method '__format__' of 'object' objects>
<slot wrapper '__ge__' of 'object' objects>
<slot wrapper '__getattribute__' of 'object' objects>
<method '__getstate__' of 'object' objects>
<slot wrapper '__gt__' of 'object' objects>
<slot wrapper '__hash__' of 'object' objects>
<function SimaProCSVImporter.__init__ at 0x0000022BBCB46020>
<built-in method __init_subclass__ of type object at 0x0000022BBDFA0C30>
<function ImportBase._

In [3]:
import sys
import inspect

def apply_logging_to_package(package, decorator):
 
     for name, obj in inspect.getmembers(package):

        if inspect.isfunction(obj):

            setattr(package, name, decorator(obj))

        elif inspect.isclass(obj):
            for method_name, method in inspect.getmembers(obj, predicate=inspect.isfunction):

                setattr(obj, method_name, decorator(method))

        elif inspect.ismodule(obj):
            for method_name, method in inspect.getmembers(obj, predicate=inspect.isfunction):

                setattr(obj, method_name, decorator(method))

apply_logging_to_package(bi, log_job_2)

In [4]:
### This is just the biosphere, but perhaps itd be better to also circle back to it

bi.remote.install_project(
    project_name="funfunfun",
    project_key="ecoinvent-3.10-biosphere", 
    projects_config={"ecoinvent-3.10-biosphere": "ecoinvent-3.10-biosphere.bw2.tar.gz"} )

Restoring project backup archive - this could take a few minutes...


'funfunfun'

In [3]:
# sorted(bw.projects)

In [5]:
bw.projects.set_current('funfunfun')

In [6]:
bw.databases

Databases dictionary with 1 object(s):
	ecoinvent-3.10-biosphere

In [2]:
np.__version__ ### make sure its not 2.0 as theres currently incompatibility issues

'1.26.4'

In [7]:
db_path = '/Users/Farhang Raymand/Downloads/ecoinvent 3.10_cutoff_ecoSpold02/datasets'      ### Takes 10.5 minutes
db_name = 'ecoinvent 3.10 cutoff -  lineage'

if 'ecoinvent 3.10 small -  lineage' in bw.databases:
    print("Database has already been imported")
else:
    ei = bi.SingleOutputEcospold2Importer(db_path, db_name)

Extracting ecospold2 files:
0% [##############################] 100% | ETA: 00:00:00 | Item ID: fffee1f5-12b7-5
Total time elapsed: 00:07:18


Title: Extracting ecospold2 files:
  Started: 09/24/2024 17:16:12
  Finished: 09/24/2024 17:23:30
  Total time elapsed: 00:07:18
  CPU %: 16.30
  Memory %: 20.42
Extracted 23523 datasets in 441.73 seconds


In [8]:
ei.apply_strategies() 
ei.statistics()
ei.write_database()
### Takes under 12 minutes fir the entire dataset (gives a non-descriptive error)

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: convert_activity_parameters_to_list
Applying strategy: add_cpc_classification_from

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:35


Title: Writing activities to SQLite3 database:
  Started: 09/24/2024 17:25:23
  Finished: 09/24/2024 17:25:59
  Total time elapsed: 00:00:35
  CPU %: 44.50
  Memory %: 21.55
Created database: ecoinvent 3.10 cutoff -  lineage


Brightway2 SQLiteBackend: ecoinvent 3.10 cutoff -  lineage

In [9]:
bw.databases

Databases dictionary with 2 object(s):
	ecoinvent 3.10 cutoff -  lineage
	ecoinvent-3.10-biosphere